### Inputs

In [ ]:
from datasets import load_dataset
import pandas as pd

### Cargar Datos

In [5]:
dataset = load_dataset("wilfredomartel/small-spanish-legal-dataset", split="train")
df = pd.DataFrame(dataset)
for index, row in df.head().iterrows():
    print(f"<----------- Fila {index} ----------->")
    print(f"QUERY: {row['query']}")
    print(f"POSITIVE: {row['pos'][0][:150]}...\n")

<----------- Fila 0 ----------->
QUERY: ¿Cuál fue la razón principal por la que la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP, presentada por Javier Jefferson Delgado Guano y otros contra la sentencia de la Sala Única Multicompetente de la Corte Provincial de Esmeraldas?
POSITIVE: La Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N° 2850-19-EP debido a que los accionantes no presentaron un argume...

<----------- Fila 1 ----------->
QUERY: ¿Por qué la Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP presentada por José Antonio Guzñay contra la sentencia de la Corte Provincial de Zamora?
POSITIVE: La Sala de Admisión de la Corte Constitucional inadmitió a trámite la acción extraordinaria de protección N°. 3090-19-EP, presentada por el señor José...

<----------- Fila 2 ----------->
QUERY: ¿Cuál fue la controversia principal en el caso N

### Preprocesamiento